In [14]:
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
# Vacination Dataset
url_vaccination = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv'
url_ages = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations-by-age-group.csv'

# Index URLs
url_index_strigency = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/stringency_index_avg.csv'
url_index_gov_response = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/government_response_index_avg.csv'
url_index_health = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/containment_health_index_avg.csv'
url_index_economic = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/economic_support_index.csv'

# Target URLs
url_cases = 'htths://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/confirmed_cases.csv'
url_deaths = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/confirmed_deaths.csv'

In [16]:
# DataFrame Index
df_strigency_raw = pd.read_csv(url_index_strigency)
df_gov_response_raw = pd.read_csv(url_index_gov_response)
df_health_raw = pd.read_csv(url_index_health)
df_economic_raw = pd.read_csv(url_index_economic)

# Vaccination
df_vaccination_raw = pd.read_csv(url_vaccination)
df_ages_raw = pd.read_csv(url_ages)


# Data Frame target
df_cases_raw = pd.read_csv(url_cases)
df_deaths_raw = pd.read_csv(url_deaths)

In [17]:
# Cleaninng Index Features

drop_columns = ['Unnamed: 0','country_code','region_code','region_name','jurisdiction']

df_gov_response = df_gov_response_raw.drop(columns = drop_columns)
df_gov_response.set_index(keys='country_name', inplace=True)
df_gov_response = df_gov_response.T
df_gov_response = df_gov_response.fillna(0)

df_health = df_health_raw.drop(columns = drop_columns)
df_health.set_index(keys='country_name', inplace=True)
df_health = df_health.T
df_health = df_health.fillna(0)

df_strigency = df_strigency_raw.drop(columns = drop_columns)
df_strigency.set_index(keys='country_name', inplace=True)
df_strigency = df_strigency.T
df_strigency = df_strigency.fillna(0)

df_economic = df_economic_raw.drop(columns = drop_columns)
df_economic.set_index(keys='country_name', inplace=True)
df_economic = df_economic.T
df_economic = df_economic.fillna(0)

# Cleaning Vaccination Features
df_vaccination = df_vaccination_raw.copy()
df_vaccination = df_vaccination[['date','location','people_vaccinated', 'people_fully_vaccinated', 'total_boosters']]


# Cleaninng Target

# Cases per country
df_cases = df_cases_raw.drop(columns = drop_columns)
df_cases.set_index(keys='country_name', inplace=True)
df_cases = df_cases.T
df_cases = df_cases.fillna(0)

# Deaths per country
df_deaths = df_deaths_raw.drop(columns = drop_columns)
df_deaths.set_index(keys='country_name', inplace=True)
df_deaths = df_deaths.T
df_deaths = df_deaths.fillna(0)

In [20]:
country = 'France'

def country_output(country):
    
    # INDEX FEATURES
    country_index = df_gov_response.copy()
    country_index = country_index[[country]]
    country_index.index.name = country
    country_index.columns = ['Gov_response']
    country_index['Containment_and_health'] = df_health[[country]]
    country_index['Stringency'] = df_strigency[[country]]
    country_index['Economics_sup'] = df_economic[[country]]

    # POPULATION VACCINATED

    country_vaccination = df_vaccination.loc[df_vaccination['location']==country]
    country_vaccination = country_vaccination.fillna(method='ffill').drop(columns = 'location')
    country_vaccination.index.name = country
    country_vaccination = country_vaccination.fillna(0)

    # TARGET

    country_target = df_cases.copy()
    country_target = country_target[[country]]
    country_target.index.name = country
    country_target.columns = ['total_cases']
    country_target['new_cases'] = country_target - country_target.shift(1)
    country_target['total_deaths'] = df_deaths[[country]]
    country_target['new_deaths'] = df_deaths[[country]] - df_deaths[[country]].shift(1)

    country_target['new_cases'].loc[country_target['new_cases'] < 0] = 0
    country_target['new_deaths'].loc[country_target['new_deaths'] < 0] = 0

    # Days no update counter

    def non_uptade(country_target):

        counter = 0
        x = 1
        while country_target['total_deaths'][-x] == 0:
            counter += 1
            x += 1

        return counter

    counter = non_uptade(country_target)

    # Last Update Data

    country_index = country_index[:-counter]
    country_vaccination = country_vaccination[:-counter]
    country_target = country_target[:-counter]
    
    return country_index, country_vaccination, country_target

country_index, country_vaccination, country_target = country_output(country)
country_index, country_vaccination, country_target 

(           Gov_response  Containment_and_health  Stringency  Economics_sup
 France                                                                    
 01Jan2020          0.00                     0.0         0.0            0.0
 02Jan2020          0.00                     0.0         0.0            0.0
 03Jan2020          0.00                     0.0         0.0            0.0
 04Jan2020          0.00                     0.0         0.0            0.0
 05Jan2020          0.00                     0.0         0.0            0.0
 ...                 ...                     ...         ...            ...
 01Sep2022         25.52                     0.0         0.0            0.0
 02Sep2022         25.52                     0.0         0.0            0.0
 03Sep2022         25.52                     0.0         0.0            0.0
 04Sep2022         25.52                     0.0         0.0            0.0
 05Sep2022         25.52                     0.0         0.0            0.0
 
 [979 rows